# Routing and Mapping in Python
One of the most important parts of the project is to determine the distances / travel times between nodes. 

We will use the open-source `openrouteservice`, which itself is based off OpenStreetMaps, to achieve this. There are alternative platforms (e.g. Google Maps), but these are more restrictive.

You will first need to register for an openrouteservice account at <https://openrouteservice.org>. Then, at the dashboard, you will need to request a token which will generate a unique key for you. You are able to query 2000 directions and 500 matrices per day.

You will also require the following packages:
* `openrouteservice` -- use `pip install openrouteservice` to install  
* `folium` for mapping -- use `conda install -c conda-forge folium` to install  
* `pandas` and `numpy`

In [3]:
ORSkey = '<redacted>'

## Mapping -- initial

Folium is Python's interface to `leaflet`. The syntax is quite a bit different to the `leaflet` package in R...

We first need to lock in a position for the map view, and then add markers as required.

In [17]:
import numpy as np
import pandas as pd
import folium
import csv

locations = pd.read_csv("WarehouseLocations.csv")


coords = locations[['Long', 'Lat']] # Mapping packages work with Long, Lat arrays
coords = coords.to_numpy().tolist() # Make the arrays into a list of lists.

# For the status quo scenario, create two empty lists to split the north and south node routes

routeListNorth = []
routeListSouth = []

# Route .csvs are given in the format 'start node','...', 'end node',
# where each line is an optimal route. Our desired format is a nested
# list containing every route.


# Read in the routes for the status quo scenario

with open('routes_statusquo.csv') as f:
    reader = csv.reader(f)
    for line in reader:
        route = [i for i in line]
        if (route[0] == "Distribution North"):
            routeListNorth.append(route)
        elif (route[0] == "Distribution South"):
            routeListSouth.append(route)

# Create an empty list for the closure scenario (no north node)

routeListClosure = []

# Read in the routes

with open('routes_closure.csv') as f:
    reader = csv.reader(f)
    for line in reader:
        route = [i for i in line]
        routeListClosure.append(route)

        
# We also created a nested list for the status quo scenario
# which did not plot north and south routes separately;
# this was chosen eventually to be the final visualisation
# method once we decided how each node would be marked.

routeList = []

with open('routes_statusquo.csv') as f:
    reader = csv.reader(f)
    for line in reader:
        route = [i for i in line]
        routeList.append(route)

# Create a dictionary of nodes and their locations

nodeDict = {locations.Store[i]: [locations.Long[i], locations.Lat[i]] for i in range(0,len(locations))}


# Folium, however, requires Lat, Long arrays - so a reversal is needed.
# coords[0] is the warehouse
m = folium.Map(location = list(reversed(coords[2])), zoom_start=10)

folium.Marker(list(reversed(coords[0])), popup = locations.Store[0], icon = folium.Icon(color = 'black')).add_to(m)

for i in range(1, len(coords)):
    if locations.Type[i] == "The Warehouse":
        iconCol = "red"
    elif locations.Type[i] == "Noel Leeming":
        iconCol = "orange"
    elif locations.Type[i] == "Distribution":
        iconCol = "black"
    folium.CircleMarker(list(reversed(coords[i])), popup = locations.Store[i], color = iconCol).add_to(m)
    
m

## Getting a route between two nodes

To get the distance between two nodes, we need to load the library `openrouteservice`.

In [4]:
import openrouteservice as ors

# Boot up client to OpenRouteService. ORSkey is your own key as a string.
client = ors.Client(key=ORSkey)



In [53]:
# Get distance in meters
route['features'][0]['properties']['summary']['distance']

36926.6

In [54]:
# Get duration in seconds
route['features'][0]['properties']['summary']['duration']

3959.8

Note that the path is stored as a list of coordinates in:
`route['features'][0]['geometry']['coordinates']`

To map this, we need to use the `PolyLine` function in Folium which draws lines between coordinates.

In [6]:
# Random hex colours were generated for each route.
# To ensure these colours were the same each time
# the code is executed, a random seed was set.

import random

# Plotting for the Northern Distribution Center routes (Status quo)

m = folium.Map(location = list(reversed(nodeDict["Distribution North"])), zoom_start=11.5, tiles = 'cartodbpositron')
    

random.seed(100)


for route in routeListNorth:
    for i in range(len(locations)):
        if locations.Store[i] in route:
            if locations.Type[i] == "The Warehouse":
                iconCol = "red"
            elif locations.Type[i] == "Noel Leeming":
                iconCol = "orange"
            elif locations.Type[i] == "Distribution":
                iconCol = "black"
            folium.CircleMarker(list(reversed(coords[i])), popup = locations.Store[i], color = iconCol).add_to(m)
            
            

    
for i in range(len(routeListNorth)):
    r = lambda: random.randint(0,255)
    iconCol = '#%02X%02X%02X' % (r(),r(),r())
    route = client.directions(
            coordinates = [nodeDict[coords] for coords in routeListNorth[i]], # Optimal routes
            # We can have more than two coords - it will generate a path between those coords in order.
            profile = 'driving-hgv', # can be driving-car, driving-hgv, etc.
            format='geojson',
            validate = False
            )    

    folium.Polygon(locations = [list(reversed(coord)) 
                              for coord in 
                              route['features'][0]['geometry']['coordinates']], 
                    color= iconCol,
                   lineWidth = 20, alpha=0.1).add_to(m)
m

In [161]:
# Plotting for the Southern Distribution Center routes (Status quo)

m = folium.Map(location = list(reversed(nodeDict["Distribution South"])), zoom_start=11.5, tiles = 'cartodbpositron')
    

random.seed(10)


for route in routeListSouth:
    for i in range(len(locations)):
        if locations.Store[i] in route:
            if locations.Type[i] == "The Warehouse":
                iconCol = "red"
            elif locations.Type[i] == "Noel Leeming":
                iconCol = "orange"
            elif locations.Type[i] == "Distribution":
                iconCol = "black"
            folium.CircleMarker(list(reversed(coords[i])), popup = locations.Store[i], color = iconCol).add_to(m)
            
            

    
for i in range(len(routeListSouth)):
    r = lambda: random.randint(0,255)
    iconCol = '#%02X%02X%02X' % (r(),r(),r())
    route = client.directions(
            coordinates = [nodeDict[coords] for coords in routeListSouth[i]], # Optimal routes
            # We can have more than two coords - it will generate a path between those coords in order.
            profile = 'driving-hgv', # can be driving-car, driving-hgv, etc.
            format='geojson',
            validate = False
            )    

    folium.Polygon(locations = [list(reversed(coord)) 
                              for coord in 
                              route['features'][0]['geometry']['coordinates']], 
                    color= iconCol,
                   lineWidth = 20, alpha=0.1).add_to(m)
m


In [16]:
# Plotting for the routes (Closure) 

m = folium.Map(location = list(reversed(nodeDict["Distribution South"])), zoom_start=11.5, tiles = 'cartodbpositron')
    

random.seed(1000)


for route in routeListClosure:
    for i in range(len(locations)):
        if locations.Store[i] in route:
            if locations.Type[i] == "The Warehouse":
                iconCol = "red"
            elif locations.Type[i] == "Noel Leeming":
                iconCol = "orange"
            elif locations.Type[i] == "Distribution":
                iconCol = "black"
            folium.CircleMarker(list(reversed(coords[i])), popup = locations.Store[i], color = iconCol).add_to(m)
            
            

    
for i in range(len(routeListClosure)):
    r = lambda: random.randint(0,255)
    iconCol = '#%02X%02X%02X' % (r(),r(),r())
    route = client.directions(
            coordinates = [nodeDict[coords] for coords in routeListClosure[i]], # Optimal routes
            # We can have more than two coords - it will generate a path between those coords in order.
            profile = 'driving-hgv', # can be driving-car, driving-hgv, etc.
            format='geojson',
            validate = False
            )    

    folium.Polygon(locations = [list(reversed(coord)) 
                              for coord in 
                              route['features'][0]['geometry']['coordinates']], 
                    color= iconCol,
                   lineWidth = 20, alpha=0.1).add_to(m)
    
f = folium.Figure(width=550, height=900)

f.add_child(m)

f


In [18]:
# Plotting for the Northern and Southern Distribution Center routes (Status quo)

m = folium.Map(location = list(reversed(nodeDict["Distribution South"])), zoom_start=11.5, tiles = 'cartodbpositron')
    

random.seed(1000)


for route in routeList:
    for i in range(len(locations)):
        if locations.Store[i] in route:
            if locations.Type[i] == "The Warehouse":
                iconCol = "red"
            elif locations.Type[i] == "Noel Leeming":
                iconCol = "orange"
            elif locations.Type[i] == "Distribution":
                iconCol = "black"
            folium.CircleMarker(list(reversed(coords[i])), popup = locations.Store[i], color = iconCol).add_to(m)
            
            

    
for i in range(len(routeList)):
    r = lambda: random.randint(0,255)
    iconCol = '#%02X%02X%02X' % (r(),r(),r())
    route = client.directions(
            coordinates = [nodeDict[coords] for coords in routeList[i]], # Optimal routes
            # We can have more than two coords - it will generate a path between those coords in order.
            profile = 'driving-hgv', # can be driving-car, driving-hgv, etc.
            format='geojson',
            validate = False
            )    

    folium.Polygon(locations = [list(reversed(coord)) 
                              for coord in 
                              route['features'][0]['geometry']['coordinates']], 
                    color= iconCol,
                   lineWidth = 20, alpha=0.1).add_to(m)
    
f = folium.Figure(width=550, height=900)

f.add_child(m)

f


OpenStreetMaps also supports the calculation of distance or duration matrices between all node pairs. 

In [16]:
matrix = client.distance_matrix(
    locations=coords,
    profile='driving-hgv',
    metrics=['distance', 'duration'],
    validate=False,
)

I have provided the output of this on Canvas so you do not need to do this yourself.

In [19]:
pd.DataFrame(matrix['durations']).to_csv('durations.csv')
pd.DataFrame(matrix['distances']).to_csv('distances.csv')